<a href="https://colab.research.google.com/github/Geraldnvs/DESAFIO_ALURA/blob/main/Desafio_Alura_%26_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai
!pip install -q -U panel
!pip install -q -U jupyter_bokeh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.2 MB/s eta 0:00:00


In [43]:
# prompt: import google genai
import google.generativeai as genai
from google.api_core import retry
import numpy as np
import pandas as pd
from pprint import pprint
import random
import re
from IPython.display import clear_output

import panel as pn
import ipywidgets as widgets
from IPython.display import display, clear_output


from google.colab import userdata

api_key = userdata.get('secret_api_key')

GOOGLE_API_KEY = api_key

genai.configure(api_key=GOOGLE_API_KEY)


from IPython.core.display import display, HTML

css_style = """
<style>
    .output {
        max-width: 200 px;
    }
    .output_area {
        width: 200px;
        max-width: 100%;
    }
    .prompt {
        min-width: fit-content;
    }
</style>
"""
display(HTML(css_style))

In [31]:
# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]



In [40]:
system_instruction = "Voce é o Mestre de Jogo (MG) em \"A Sombra sobre Valdar\" tem a responsabilidade de conduzir a narrativa e gerenciar o sistema de jogo de forma que enriqueça a experiência dos jogadores. Ele cria o cenário, narra as consequências das ações dos jogadores e controla os adversários seguindo as regras no estilo de Dungeons & Dragons.\n\nApresentação de Opções pelo Mestre de Jogo:\n\nDurante o jogo, sempre que os jogadores tiverem que fazer uma escolha, o MG apresentará as opções de forma clara e estruturada. Isso é feito para facilitar a decisão e manter o jogo \n\nOs jogadores então discutem e decidem qual opção seguir, facilitando o processo de decisão e mantendo o jogo fluindo de forma dinâmica.\n\nGerenciamento de Combate e Diálogos Objetivos:\n\nEm combate, o MG segue as mecânicas detalhadas para rolagens de ataque e dano. Além disso, todos os diálogos conduzidos pelo MG, especialmente em situações de negociação ou diálogo com NPCs, serão curtos e objetivos, mantendo o ritmo do jogo e a clareza das informações.\nPapel Além do Combate:\n\nO MG utiliza diálogos e descrições para enriquecer a trama e orientar os jogadores através das complicações do enredo, sempre mantendo a clareza e a brevidade na comunicação. Isso inclui a administração das consequências das escolhas dos jogadores, desenvolvendo a história de forma coesa e envolvente.\n voce sempre fala em portugues do brasil"

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

chat = model.start_chat(history=[])



In [ ]:
prompt = "podemos iniciar o jogo, me pergunte quantos aventureiros somos e quais nossas raças e classes e quais equipamentos utilizamos."

prompt_parts = [
"""
Sempre que apresentar um contexo marque o inicio e o fim com $$
sempre que apresentar opçoes para o jogador faça marcando o inicio e o fim da opçao comm os simbolos ##

Entrada: Descreva opções para um jogador que encontrou um animal ferido na floresta.
$$Durante a exploração de uma densa floresta antiga, um dos jogadores ouve um fraco gemido de dor proveniente de um denso matagal próximo a um riacho. Ao investigar a origem do som, o jogador descobre um jovem cervo preso em uma armadilha de caçador. O animal está visivelmente ferido e assustado, com uma das pernas presas e feridas evidentes. O sol começa a se pôr, e o jogador precisa tomar uma decisão rápida sobre como agir diante desta situação delicada.Durante a exploração de uma densa floresta antiga, um dos jogadores ouve um fraco gemido de dor proveniente de um denso matagal próximo a um riacho. Ao investigar a origem do som, o jogador descobre um jovem cervo preso em uma armadilha de caçador. O animal está visivelmente ferido e assustado, com uma das pernas presas e feridas evidentes. O sol começa a se pôr, e o jogador precisa tomar uma decisão rápida sobre como agir diante desta situação delicada.$$
## Cuidar do Animal: Você tenta cuidar do animal usando suas habilidades de sobrevivência. **Agora você precisa jogar os dados, boa sorte!** (Rolagem de D20 + bônus de Sabedoria). Se for bem-sucedido, o animal se recuperará e poderá se tornar um aliado.##
## Ignorar o Animal:** Você escolhe não intervir e continua sua jornada. Não é necessário rolar os dados, mas essa escolha pode afetar a percepção dos NPCs sobre seu caráter.##
## Usar Magia de Cura:** Você decide usar uma magia de cura no animal. **Agora você precisa jogar os dados, boa sorte!** (Rolagem de D20 + bônus de Conjuração). Se for bem-sucedido, o animal será curado instantaneamente. ##
## Levar o Animal ao Veterinário Local:** Você opta por levar o animal a um especialista na cidade mais próxima. **Agora você precisa jogar os dados, boa sorte!** (Rolagem de D20 + bônus de Carisma para convencer o veterinário a ajudar). Se for bem-sucedido, o animal será curado e você ganhará um novo amigo. ##


""",
]
def roll_d20():
      """Retorna o resultado de um lançamento de um dado de 20 lados (d20)."""
      return random.randint(1, 20)
def callback(event):
      choice = radio_group.value
      dado = roll_d20()
      result_display.value = f"{choice}. Dado rolado e o valor foi {dado}"

prompt_parts_atual = [prompt_parts[0] + "\n" + prompt,]

response = model.generate_content(prompt)
print(response.text)
prompt = input("Digite sua resposta ou 'fim' para sair: ")
while prompt != "fim":
  clear_output()
  response = chat.send_message(prompt+" se precisar de valor do dado é " + str(roll_d20()))
  response_text = response.text
  print(response_text)
  prompt = input("Digite sua resposta ou 'fim' para sair: ")





Excelente! Um cavaleiro de armadura pesada, empunhando escudo e espada longa. Um verdadeiro arauto da justiça, pronto para defender os inocentes e fazer valer a lei! ⚔️🛡️

Com um valor 18 nos dados, a sorte parece estar ao seu lado! Conte-me mais sobre seu cavaleiro:

* Qual o nome dele e o que o impulsiona a enfrentar os perigos de Valdar? 
* Qual a ordem de cavalaria que ele serve, ou ele segue seu próprio código de ética? 
* O que o motiva a se aventurar em Valdar? Uma missão específica, a busca por glória, ou talvez um chamado maior? 🤔

Quanto mais detalhes você me der sobre seu personagem, mais imersiva e emocionante será a sua aventura em Valdar! 😉 



In [21]:
prompt+" se precisar de valor do dado é " + str(roll_d20())

'podemos iniciar o jogo, me pergunte quantos aventureiros somos e quais nossas raças e classes e quais equipamentos utilizamos. se precisar de valor do dado é 2'